<a href="https://colab.research.google.com/github/AmitRLohiya/BigMartSales_AnalyticsVidhya/blob/main/Practise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**#Used Fancy Impute and XGB Classfier for Missing Value Impuatation**

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [71]:
#Importing Training Data and Test Data
bigmarttrain = pd.read_csv("/content/drive/MyDrive/Machine_Learning/bigmart_train.csv")
bigmarttest = pd.read_csv('/content/drive/MyDrive/Machine_Learning/bigmart_test.csv')

In [72]:
bigmarttrain.head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [73]:
list_object = list(bigmarttrain.select_dtypes(include=np.object).columns)

for i in list_object:
    print("Column Name \"{}\"".format(i))
    print("No.of Unique Values {}".format(bigmarttrain[i].nunique()))

Column Name "Item_Identifier"
No.of Unique Values 1559
Column Name "Item_Fat_Content"
No.of Unique Values 5
Column Name "Item_Type"
No.of Unique Values 16
Column Name "Outlet_Identifier"
No.of Unique Values 10
Column Name "Outlet_Size"
No.of Unique Values 3
Column Name "Outlet_Location_Type"
No.of Unique Values 3
Column Name "Outlet_Type"
No.of Unique Values 4


In [74]:
for i in list_object:
    if i == "Item_Identifier":
        None
    else:
        print("Column Name \"{}\"".format(i))
        print("No.of Unique Values {}".format(bigmarttrain[i].unique()))
        print("\n")
        
#Irregularities in Item_Fat_Content -> Low Fat - ['low fat', 'LF'], Regular - ['reg', 'Regular']

#Grouping Outlet_Identifiers with Outlet Type can provide a brief if Outlet_Identifiers have an association with an Outlet_type

#Outlet_Size has a value nan

Column Name "Item_Fat_Content"
No.of Unique Values ['Low Fat' 'Regular' 'low fat' 'LF' 'reg']


Column Name "Item_Type"
No.of Unique Values ['Dairy' 'Soft Drinks' 'Meat' 'Fruits and Vegetables' 'Household'
 'Baking Goods' 'Snack Foods' 'Frozen Foods' 'Breakfast'
 'Health and Hygiene' 'Hard Drinks' 'Canned' 'Breads' 'Starchy Foods'
 'Others' 'Seafood']


Column Name "Outlet_Identifier"
No.of Unique Values ['OUT049' 'OUT018' 'OUT010' 'OUT013' 'OUT027' 'OUT045' 'OUT017' 'OUT046'
 'OUT035' 'OUT019']


Column Name "Outlet_Size"
No.of Unique Values ['Medium' nan 'High' 'Small']


Column Name "Outlet_Location_Type"
No.of Unique Values ['Tier 1' 'Tier 3' 'Tier 2']


Column Name "Outlet_Type"
No.of Unique Values ['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']




In [75]:
#Look at Test data on Object Columns

list_object = list(bigmarttest.select_dtypes(include=np.object).columns)
for i in list_object:
    if i == "Item_Identifier":
        None
    else:
        print("Column Name \"{}\"".format(i))
        print("No.of Unique Values {}".format(bigmarttest[i].unique()))
        print("\n")

#Test Data also had same Irregularities

Column Name "Item_Fat_Content"
No.of Unique Values ['Low Fat' 'reg' 'Regular' 'LF' 'low fat']


Column Name "Item_Type"
No.of Unique Values ['Snack Foods' 'Dairy' 'Others' 'Fruits and Vegetables' 'Baking Goods'
 'Health and Hygiene' 'Breads' 'Hard Drinks' 'Seafood' 'Soft Drinks'
 'Household' 'Frozen Foods' 'Meat' 'Canned' 'Starchy Foods' 'Breakfast']


Column Name "Outlet_Identifier"
No.of Unique Values ['OUT049' 'OUT017' 'OUT010' 'OUT027' 'OUT046' 'OUT018' 'OUT045' 'OUT019'
 'OUT013' 'OUT035']


Column Name "Outlet_Size"
No.of Unique Values ['Medium' nan 'Small' 'High']


Column Name "Outlet_Location_Type"
No.of Unique Values ['Tier 1' 'Tier 2' 'Tier 3']


Column Name "Outlet_Type"
No.of Unique Values ['Supermarket Type1' 'Grocery Store' 'Supermarket Type3'
 'Supermarket Type2']




In [76]:
#Look at the Numeric Data
bigmarttrain.describe()
#Irregularity in Counts of Item_Weight Columns

#Item_Visibility has a minimum value of "0" -> Equals to Nan Value as Visibility cannot be negative Value

#New Feature Item_Outlet_Age can be created using Outlet_Establishment Year

#Item_Outlet_Sales are highly Varying on Scale from 33.29 to 13086.96

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [77]:
#Looking at Null Values in the data

null = pd.DataFrame(bigmarttrain.isnull().sum(), columns = ['Count of Null Values'])
(null['Count of Null Values'].apply(lambda x: (x/bigmarttrain.shape[0])*100))
# null.sort_values(by = 'Percent_Of_Null', ascending = False)

Item_Identifier               0.000000
Item_Weight                  17.165317
Item_Fat_Content              0.000000
Item_Visibility               0.000000
Item_Type                     0.000000
Item_MRP                      0.000000
Outlet_Identifier             0.000000
Outlet_Establishment_Year     0.000000
Outlet_Size                  28.276428
Outlet_Location_Type          0.000000
Outlet_Type                   0.000000
Item_Outlet_Sales             0.000000
Name: Count of Null Values, dtype: float64

In [78]:
from fancyimpute import IterativeImputer

In [79]:
mice = IterativeImputer()

In [80]:
#Adding Identifier COlumns to Train and Test Data
bigmarttrain['Type'] = "Train"
bigmarttest['Type'] = "Test"

In [81]:
#Dropping the Target Column and Combining the DataFrames
combinedf = pd.concat([bigmarttrain.drop('Item_Outlet_Sales', axis = 1), bigmarttest], axis=0)

In [82]:
#Segregating Numeric Data to Impute Missing Values
numeric_data = combinedf.select_dtypes(include = np.number)
categorical_data = combinedf.select_dtypes(include = np.object)

In [83]:
#Initialize Visibility '0' with np.Nan Values as Visibility cannot be 0
numeric_data['Item_Visibility'] = np.where(numeric_data['Item_Visibility'] == 0 , np.NaN, numeric_data['Item_Visibility'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [84]:
combinedf.shape

(14204, 12)

In [85]:
#Initializing Imputer with MICE
mice = IterativeImputer()

In [86]:
numeric_data.isnull().sum().sort_values(ascending = False)

Item_Weight                  2439
Item_Visibility               879
Outlet_Establishment_Year       0
Item_MRP                        0
dtype: int64

In [87]:
numericdf = mice.fit_transform(numeric_data)

In [88]:
numericdf = pd.DataFrame(numericdf, columns = list(numeric_data.columns))
numericdf.isnull().sum().sort_values(ascending = False)

Outlet_Establishment_Year    0
Item_MRP                     0
Item_Visibility              0
Item_Weight                  0
dtype: int64

In [89]:
#Lets Have a look wtowards the deviation of Mean After Imputing the Missing Values with Mice Algp
numericdf.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
count,14204.000000,14204.000000,14204.000000,14204.000000
mean,12.792613,0.070305,141.004977,1997.830681
std,4.234735,0.048604,62.086938,8.371664
min,4.555000,0.003575,31.290000,1985.000000
25%,9.300000,0.033143,94.012000,1987.000000
50%,12.748001,0.062315,142.247000,1999.000000
75%,16.000000,0.094037,185.855600,2004.000000
max,21.350000,0.328391,266.888400,2009.000000


After using Mice Imputer the Mean is approximately Similar to the Mean before Imputation -> Implication Data has not been distorted

In [90]:
numericdf.reset_index(inplace=True)
categorical_data.reset_index(inplace = True)
combinedf = pd.concat([numericdf, categorical_data], axis = 1)

In [91]:
combinedf[['Item_Identifier', 'Item_Type']].head(10)
#FD - Food , DR - Drinks and NC - Non Consumable goods

,Item_Identifier,Item_Type
0,FDA15,Dairy
1,DRC01,Soft Drinks
2,FDN15,Meat
3,FDX07,Fruits and Vegetables
4,NCD19,Household
5,FDP36,Baking Goods
6,FDO10,Snack Foods
7,FDP10,Snack Foods
8,FDH17,Frozen Foods
9,FDU28,Frozen Foods


In [92]:
combinedf['codes'] = combinedf['Item_Identifier'].apply(lambda x: x[0:2])
combinedf['codes'].head()
combinedf.drop(['Item_Identifier'], axis = 1, inplace=True)
combinedf.head(2)

,index,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,index,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Type,codes
0,0,9.30,0.016047,249.8092,1999.0,0,Low Fat,Dairy,OUT049,Medium,Tier 1,Supermarket Type1,Train,FD
1,1,5.92,0.019278,48.2692,2009.0,1,Regular,Soft Drinks,OUT018,Medium,Tier 3,Supermarket Type2,Train,DR


In [93]:
categorical_data=combinedf.select_dtypes(include = np.object)
numeric_data = combinedf.select_dtypes(include = np.number)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
categoricaldf = categorical_data.drop('Outlet_Size', axis = 1)
categoricaldf = categoricaldf.apply(le.fit_transform)
categoricaldf['Outlet_Size'] = categorical_data.Outlet_Size
#Categorical Data is Processed

In [98]:
categoricaldf.head(2)

,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Location_Type,Outlet_Type,Type,codes,Outlet_Size
0,1,4,9,0,1,1,1,Medium
1,2,14,3,2,2,1,0,Medium


In [101]:
numeric_data.drop('index', axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [102]:
numeric_data.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
0,9.30,0.016047,249.8092,1999.0
1,5.92,0.019278,48.2692,2009.0
2,17.50,0.016760,141.6180,1999.0
3,19.20,0.068745,182.0950,1998.0
4,8.93,0.077558,53.8614,1987.0


In [113]:
numeric_data['Age'] = 2021-numeric_data['Outlet_Establishment_Year']
numeric_data.drop('Outlet_Establishment_Year', axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [114]:
#Processing Numeric Data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [115]:
numericdf = scaler.fit_transform(numeric_data)
numericdf = pd.DataFrame(numericdf, columns = list(numeric_data))
numericdf.head(2)

,Item_Weight,Item_Visibility,Item_MRP,Age
0,0.282525,0.038399,0.927507,0.416667
1,0.081274,0.048346,0.072068,0.000000


In [94]:
categoricaldf.isnull().sum()

Item_Fat_Content           0
Item_Type                  0
Outlet_Identifier          0
Outlet_Location_Type       0
Outlet_Type                0
Type                       0
codes                      0
Outlet_Size             4016
dtype: int64

In [122]:
combinedf = pd.concat([categoricaldf, numericdf], axis = 1)

In [123]:
train_df = combinedf[~pd.isnull(combinedf['Outlet_Size'])]
test_df  = combinedf[pd.isnull(combinedf['Outlet_Size'])]

In [124]:
test_df.shape

(4016, 12)

In [125]:
train_df.shape

(10188, 12)

In [127]:
train_df.head(2)

,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Location_Type,Outlet_Type,Type,codes,Outlet_Size,Item_Weight,Item_Visibility,Item_MRP,Age
0,1,4,9,0,1,1,1,Medium,0.282525,0.038399,0.927507,0.416667
1,2,14,3,2,2,1,0,Medium,0.081274,0.048346,0.072068,0.000000


In [128]:
x = train_df.drop('Outlet_Size', axis = 1)
y = train_df.Outlet_Size

In [142]:
test_df.drop('Outlet_Size', axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [131]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(x,y)

XGBClassifier(objective='multi:softprob')

In [133]:
trainpred = xgb.predict(x)

In [132]:
from sklearn.metrics import classification_report

In [134]:
print(classification_report(y, trainpred))

              precision    recall  f1-score   support

        High       1.00      1.00      1.00      1553
      Medium       1.00      1.00      1.00      4655
       Small       1.00      1.00      1.00      3980

    accuracy                           1.00     10188
   macro avg       1.00      1.00      1.00     10188
weighted avg       1.00      1.00      1.00     10188



In [143]:
test_df.head()

,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Location_Type,Outlet_Type,Type,codes,Item_Weight,Item_Visibility,Item_MRP,Age
3,2,6,0,2,0,1,1,0.871986,0.200638,0.640093,0.458333
8,2,5,7,1,1,1,1,0.693361,0.040369,0.278791,0.291667
9,2,5,2,1,1,1,1,0.871986,0.279773,0.664399,0.083333
25,1,9,2,1,1,1,2,0.502828,0.296514,0.062038,0.083333
28,2,4,0,2,0,1,1,0.081572,0.486096,0.060351,0.458333


In [144]:
testpred = xgb.predict(test_df)
len(testpred)

4016

In [147]:
pd.Series(testpred).isnull().sum()

0

In [148]:
test_df['Outlet_Size'] = testpred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [149]:
test_df.Outlet_Size.value_counts()

Small    3091
High      925
Name: Outlet_Size, dtype: int64

In [150]:
test_df.isnull().sum()

Item_Fat_Content        0
Item_Type               0
Outlet_Identifier       0
Outlet_Location_Type    0
Outlet_Type             0
Type                    0
codes                   0
Item_Weight             0
Item_Visibility         0
Item_MRP                0
Age                     0
Outlet_Size             0
dtype: int64

In [151]:
combinedf = pd.concat([train_df, test_df], axis = 0)
combinedf.head()

,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Location_Type,Outlet_Type,Type,codes,Outlet_Size,Item_Weight,Item_Visibility,Item_MRP,Age
0,1,4,9,0,1,1,1,Medium,0.282525,0.038399,0.927507,0.416667
1,2,14,3,2,2,1,0,Medium,0.081274,0.048346,0.072068,0.000000
2,1,10,9,0,1,1,1,Medium,0.770765,0.040593,0.468288,0.416667
4,1,9,1,2,1,1,2,High,0.260494,0.227769,0.095805,0.916667
5,2,0,3,2,2,1,1,Medium,0.347723,0.190014,0.085361,0.000000


In [155]:
Outlet_Size = le.fit_transform(combinedf.Outlet_Size)
combinedf['Outlet_Size'] = pd.Series(Outlet_Size)

In [158]:
combinedf.Type.value_counts()
#1->Train Data
#0->Test Data

1    8523
0    5681
Name: Type, dtype: int64